In [1]:
import scanpy as sc
import pandas as pd
import numpy as np
import decoupler as dc
import os
import gseapy
import matplotlib.pyplot as plt
import squidpy as sq
import pickle

In [9]:
def merge_adata(adata, merge_dict):
    adata.obs['merge_cluster'] = adata.obs['leiden'].map(merge_dict)
    sc.tl.rank_genes_groups(
        adata, groupby="merge_cluster", method="wilcoxon", key_added="wilcoxon_merged")
    sc.tl.filter_rank_genes_groups(
        adata,
        key="wilcoxon_merged",
        key_added="wilcoxon_merged_filtered")
    # plot
    sc.pl.rank_genes_groups_dotplot(adata, key = 'wilcoxon_merged_filtered',
                                standard_scale="var", n_genes=6, show=False)
    sc.pl.rank_genes_groups_dotplot(adata, key = 'wilcoxon_merged',
                                standard_scale="var", n_genes=6, show=False)
    sc.pl.umap(adata, color = 'merge_cluster')
    sq.pl.spatial_scatter(adata, 
              shape = None, 
              color=['merge_cluster'],
              wspace = 0.1)
    
    return adata
    
def final_plot(adata, cell_type_dict, genemarkers, sample):
    adata.obs['group'] = adata.obs['merge_cluster'].map(cell_type_dict)
    adata.obs['group'] = adata.obs['group'].astype('category').cat.set_categories(list(genemarkers.keys()))
    
    sc.pl.dotplot(adata, var_names=genemarkers, groupby = 'group', 
                        standard_scale="var", show=False)
    sc.pl.umap(adata, color='group', show=False, title = sample)

    sq.pl.spatial_scatter(adata, 
          shape = None, 
          color=['group'],
          wspace = 0.1, title = sample)
    

def get_top_genes(adata):
    tmp_rank_df = sc.get.rank_genes_groups_df(adata, group=adata.obs.merge_cluster.unique(), key= 'wilcoxon_merged_filtered')
    tmp_rank_df = tmp_rank_df[~tmp_rank_df.names.isna()].groupby('group').apply(lambda x: x.nlargest(6, 'scores')) 
    return tmp_rank_df 
    

In [ ]:
SAMPLE_ID_merge = {
    '0':'0',
    '1':'0',
    '2':'0',
    '3':'0',
    '4':'1',
    '5':'2',
    '6':'0',
    '7':'0',

}

with open('/SAMPLE_ID_.pkl', 
          'rb') as f:
    sdata_primary = pickle.load(f)   
sdata_primary = merge_adata(sdata_primary, SAMPLE_ID_merge)

In [ ]:
SAMPLE_ID_cell_type = {
    '0':'Malignant',
    '2':'Macrophage',
    '1': 'Fibroblast',
}
SAMPLE_ID_markers = {
    'Malignant':['EPCAM', 'EGFR', 'ERBB2', 'ERBB3'],
    'Macrophage':['APOE','MPEG1', 'CD163'],
    'Fibroblast':['PLVAP', 'RGS5', 'DCN', 'SPARC'], 
    
}
final_plot(sdata_primary, SAMPLE_ID_cell_type, SAMPLE_ID_markers, 'SAMPLE_ID')